In [38]:
import numpy as np
import pandas as pd

from sklearn import datasets 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from RandomForestExplorer.find_rules import find_rules
from RandomForestExplorer.find_rules import find_rules_on_samples

In [39]:
iris_data = datasets.load_iris()
feature_names = iris_data['feature_names']

In [47]:
X = iris_data['data'].astype(np.float32)
y = iris_data['target']

#to drop class with label y=2
mask = y == 2
X = X[~mask].astype(np.float32)
y = y[~mask]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

seed = 0 # for reproducibity
n_estimators=100
rf = RandomForestClassifier(n_estimators=n_estimators, random_state=seed).fit(X_train, y_train)

In [48]:
y_test_pred = rf.predict(X_test)
accuracy_score(y_test, y_test_pred)

1.0

In [49]:
sample_id = 0
rules, is_pred_correct = find_rules(rf, X_test[sample_id], y_test[sample_id], min_support=0.4)

In [50]:
rules

,support,itemsets
0,0.50,"((2, 2.449999988079071, 1))"
1,0.44,"((3, 0.75, 1))"


In [51]:
is_pred_correct

True

## Get rules for set of observations in the same class

In [52]:
class_id = 0

y_true = class_id
X_train_in_class = X_train[y_train == y_true]


df = find_rules_on_samples(rf, X_train_in_class, y_true)
df

,support,itemsets
0,0.504435,"((2, 2.449999988079071, 1))"
1,0.442060,"((3, 0.75, 1))"


In [53]:
class_id = 1

y_true = class_id
X_train_in_class = X_train[y_train == y_true]


df = find_rules_on_samples(rf, X_train_in_class, y_true)
df

,support,itemsets
0,0.497279,"((2, 2.449999988079071, 0))"
1,0.464050,"((3, 0.75, 0))"


In [54]:
X_encoded = X[:,[2,3]]

# we use the discovered rules to encode the data

# column 0 of `X_test_encoded`, which is feature with index 2 in X_test
mask = X_encoded[:, 0] < 2.45
X_encoded[mask, 0] = 1
X_encoded[~mask, 0] = 0

# column 1 of `X_test_encoded`, which is feature with index 3 in X_test
mask = X_encoded[:, 1] <  0.75
X_encoded[mask, 1] = 1
X_encoded[~mask, 1] = 0

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=0)

seed = 0 # for reproducibity
rf = RandomForestClassifier(n_estimators=10, random_state=seed).fit(X_train, y_train)
y_test_pred = rf.predict(X_test)

accuracy_score(y_test, y_test_pred)

1.0